# Processing of Files for the MANCHI Station

Test notebook for data processing

## Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import gnssvod as gv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pdb
import importlib
import zipfile
import os
import xarray as xr
import glob
import shutil
from gnssvod.hemistats.hemistats import hemibuild
#import georinex as gr
#import qgrid as interactive table 
from matplotlib.collections import PatchCollection
import matplotlib.dates as mdates
import shutil
from matplotlib import rcParams
from datetime import datetime, timedelta

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Presets

### Time intervals

In [19]:
# Define the start day
startday = pd.to_datetime('25-04-2025', format='%d-%m-%Y')
# Generate a range of datetime values
timeintervals=pd.interval_range(start=startday, periods=10, freq='D', closed='left')
timeintervals

IntervalIndex([[2025-04-25 00:00:00, 2025-04-26 00:00:00),
               [2025-04-26 00:00:00, 2025-04-27 00:00:00),
               [2025-04-27 00:00:00, 2025-04-28 00:00:00),
               [2025-04-28 00:00:00, 2025-04-29 00:00:00),
               [2025-04-29 00:00:00, 2025-04-30 00:00:00),
               [2025-04-30 00:00:00, 2025-05-01 00:00:00),
               [2025-05-01 00:00:00, 2025-05-02 00:00:00),
               [2025-05-02 00:00:00, 2025-05-03 00:00:00),
               [2025-05-03 00:00:00, 2025-05-04 00:00:00),
               [2025-05-04 00:00:00, 2025-05-05 00:00:00)],
              dtype='interval[datetime64[ns], left]')

## Process Rinex Files

In [15]:
import gnssvod as gv

pattern = {
    'MANCHI': r'C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\Last\MANCHI_20250420223245\MANCHI_raw_20250420223245_RINEX_3_03\MANCHI_raw_20250420223245.25O'
}

result = gv.preprocess(pattern, interval='15s', keepvars=['S?','S??'], 
                       outputresult=True)

obs = result['MANCHI'][0]
obs


Created a temporary directory at C:\Users\varga\AppData\Local\Temp\tmp5lcdpkra
C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\Last\MANCHI_20250420223245\MANCHI_raw_20250420223245_RINEX_3_03\MANCHI_raw_20250420223245.25O exists | Reading...
Observation file  C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\Last\MANCHI_20250420223245\MANCHI_raw_20250420223245_RINEX_3_03\MANCHI_raw_20250420223245.25O  is read in 7.42 seconds.
Processing 95433 individual observations
Calculating Azimuth and Elevation
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmp5lcdpkra\GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3.gz: 1.05MB [00:03, 317kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3.gz
C:\Users\varga\AppData\Local\Temp\tmp5lcdpkra\GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3 file is read in 5.44 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmp5lcdpkra\GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3.gz: 1.04MB [00:03, 320kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3.gz
C:\Users\varga\AppData\Local\Temp\tmp5lcdpkra\GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3 file is read in 12.57 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmp5lcdpkra\GFZ0MGXRAP_20251120000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251120000_01D_05M_ORB.SP3.gz

...

 | Download completed for GFZ0MGXRAP_20251100000_01D_30S_CLK.CLK.gz
C:\Users\varga\AppData\Local\Temp\tmp5lcdpkra\GFZ0MGXRAP_20251100000_01D_30S_CLK.CLK file is read in 11.30 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmp5lcdpkra\GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK.gz: 4.80MB [00:04, 1.09MB/s]                            


 | Download completed for GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK.gz
C:\Users\varga\AppData\Local\Temp\tmp5lcdpkra\GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK file is read in 31.23 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmp5lcdpkra\GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK.gz: 4.80MB [00:03, 1.30MB/s]                            


 | Download completed for GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK.gz
C:\Users\varga\AppData\Local\Temp\tmp5lcdpkra\GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK file is read in 12.30 seconds
SP3 interpolation is done in 48.78 seconds
Removed the temporary directory at C:\Users\varga\AppData\Local\Temp\tmp5lcdpkra


In [17]:
obs = result['MANCHI'][0]
obs.epoch

datetime.date(2025, 4, 21)

In [20]:
obs.observation

S1C  S1X   S2C   S2I  S2X  S7I  S7X     Azimuth  \
Epoch               SV                                                      
2025-04-20 22:33:30 C23   NaN  NaN   NaN  20.0  NaN  NaN  NaN  166.650442   
                    C27   NaN  NaN   NaN  48.0  NaN  NaN  NaN  -18.417633   
                    C28   NaN  NaN   NaN  40.0  NaN  NaN  NaN   51.844026   
                    C32   NaN  NaN   NaN  36.0  NaN  NaN  NaN -119.403873   
                    C33   NaN  NaN   NaN  17.0  NaN  NaN  NaN   30.431649   
...                       ...  ...   ...   ...  ...  ...  ...         ...   
2025-04-21 22:33:00 R08  27.0  NaN  26.0   NaN  NaN  NaN  NaN   16.126631   
                    R12  19.0  NaN   NaN   NaN  NaN  NaN  NaN -167.267077   
                    R22  42.0  NaN  33.0   NaN  NaN  NaN  NaN   92.813625   
                    R23  33.0  NaN   NaN   NaN  NaN  NaN  NaN         NaN   
                    R24  37.0  NaN  29.0   NaN  NaN  NaN  NaN  -23.953310   

                         Elevation  
Epoch               SV              
2025-04-20 22:33:30 C23  10.878548  
                    C27  39.076360  
                    C28  63.342841  
                    C32  45.994334  
                    C33  18.276292  
...                            ...  
2025-04-21 22:33:00 R08  39.955684  
                    R12   3.436013  
                    R22  34.655140  
                    R23        NaN  
                    R24   8.772009  

[95433 rows x 9 columns]

## Save NCs

In [21]:

pattern = {
    'MANCHI': r'C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\Last\MANCHI_20250420223245\MANCHI_raw_20250420223245_RINEX_3_03\MANCHI_raw_20250420223245.25O'
}
outputdir = {
    'MANCHI': r'C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\nc\\'
}
result = gv.preprocess(
    pattern,
    interval='15s',
    keepvars=['S?','S??'],   # SNR típicamente
    outputdir=outputdir,
    outputresult=True,
    # approx_position=approx_position
)

Created a temporary directory at C:\Users\varga\AppData\Local\Temp\tmparjp82yl
Could not find any files matching the pattern C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\nc\\*.nc
C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\Last\MANCHI_20250420223245\MANCHI_raw_20250420223245_RINEX_3_03\MANCHI_raw_20250420223245.25O exists | Reading...
Observation file  C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\Last\MANCHI_20250420223245\MANCHI_raw_20250420223245_RINEX_3_03\MANCHI_raw_20250420223245.25O  is read in 3.66 seconds.
Processing 95 433 individual observations
Calculating Azimuth and Elevation
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3.gz: 1.05MB [00:03, 340kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3.gz
C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3 file is read in 7.27 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3.gz: 1.04MB [00:03, 330kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3.gz
C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3 file is read in 13.72 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251120000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251120000_01D_05M_ORB.SP3.gz

...

 | Download completed for GFZ0MGXRAP_20251100000_01D_30S_CLK.CLK.gz
C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251100000_01D_30S_CLK.CLK file is read in 4.02 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK.gz: 4.80MB [00:03, 1.30MB/s]                            


 | Download completed for GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK.gz
C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK file is read in 31.18 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK.gz: 4.80MB [00:04, 1.04MB/s]                            


 | Download completed for GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK.gz
C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK file is read in 4.97 seconds
SP3 interpolation is done in 33.78 seconds
Saved 95 433 individual observations in C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\nc\MANCHI_raw_20250420223245.nc
Removed the temporary directory at C:\Users\varga\AppData\Local\Temp\tmparjp82yl
